# Detekce, popis a klasifikace obličeje z RGB obrazu
Druhá část cvičení se zaměří na detekci obličeje pomocí tradičních metod zpracování obrazu a následně na popis a klasifikaci detekovaného obličeje pomocí hlubokých neuronových sítí. Soudobé přístupy oba tyto úkoly typicky spojují do jedné sítě, ale o těch se zmíníme až na konci...


## Detekce obličeje
Prvním úkolem je ve vstupním obrazu detekovat umístění obličeje nebo obličejů. Jednak nám to umožní vyfiltrovat nedůležité části obrazu před dalším zpracováním a typicky tak zpracování urychlit a zpřesnit, ale především nám to pomůže při výskytu více obličejů v jednom obrázku. Metody popisu totiž chceme spouštět zaručeně jen na jednom obličeji...

Jenže, jak na to? Zkusme si představit jak vypadá typický obličej. Oči a obočí jsou tmavší než tváře, stejně tak jsou tmavší než nos. Jenže modelovat explicitně podobu obličeje je poměrně pracné. Především, když můžeme použít hrubou sílu a nechat si vymodelovat typický obličej ne na úrovni pixelů, ale na úrovni rozdílů intenzit vhodně zvolených oblastí napříč rozlišeními.

K tomu je možné použít například Haarovu funkci všemožně naškálovanou a narotovanou jak je podrobněji popsáno v článku Paula Violy a Michaela Jonese: "Rapid Object Detection using a Boosted Cascade of Simple Features" a klasifikátor, který bude detekovat ty oblasti obrázku, které obsahují předučenou kombinaci rozdílů intenzit podle vybraných funkcí.

<img src="images/haar_wavelet.svg" width=40% style="float:left"> <img src="images/haar_application.png" width=40%>
<div style="clear:both"/>

Pro učení klasifikátoru je zapotřebí algoritmu dodat velké množství obrázků obličeje (a obrázků, které obličej neobsahují), na kterých proběhne, zjednodušeně řečeno, určení hodnot všech Haarových funkcí a výběr funkcí (a prahů) vhodných pro rozlišení zda obrázek obsahuje nebo neobsahuje obličej. Tím vznikne model, který je při dostatečných schopnostech generalizace možné následně používat pro detekci i neznámých obličejů. Model zde z dat učit nebudeme, použijeme model již natrénovaný pro detekci obličeje dívajícího se přímo do kamery, který je distribuovaný s OpenCV. Pro jistotu jej ale přikládáme přímo do složky s notebookem.

### Importy knihoven a modelu

In [ ]:
%run ../svz.ipynb
face_cascade = cv2.CascadeClassifier('models/haarcascade_frontalface_default.xml')

### Příprava obrázku a detekce
#### 1) Načtěte pomocí knihovny OpenCV obrázek `who_is_this.jpg`, převeďte do RGB pro další zpracování a do odstínů šedi pro detekci Haarovou kaskádou.

In [ ]:
image =  ###
image_rgb = ...(image) ###
image_gray = ...(image) ###

plot_images(image_rgb, image_gray)

#### 2) Detekujte obličeje. Uvedené parametry funkce detectMultiScale specifikují úroveň škálování a citlivost filtru.
Pro další práci zachovejte jen první detekovaný obličej, vizualizujte jeho detekci nakreslením obdélníku přes fotografii. **HINT:** *Detekovaný obličej se z pohledu opencv tváří jako Rectangle*.

In [ ]:
# Detects faces
faces = face_cascade.detectMultiScale(image_gray, 1.3, 5)
face_rectangle =  ###

# Refactors face variable
(x,y,w,h) =  ###

# Draws rectangle
viz = image_rgb.copy()
cv2.rectangle(viz, (x,y), (x + w, y + h), (255,0,0), 2)

plot_images(viz)

#### 3) Oblast obličeje ořízněte, zkontrolujte zda je výřez čtvercový, případně upravte.

In [ ]:
cropped = ...(image_rgb, ?, ?, ?, ?) ###
resolution = cropped.shape[0:2]

ratio =  ###
if ratio == 1:
    print('Je to čtverec')
else:
    raise Exception('Není to čtverec, upravte')

---
## Popis a klasifikace obličeje
Nyní když máme obličej nahrubo označený, zajímalo by nás kdo se za ním skrývá.

Opět je pro tyto úlohy používáno velké množství hrubé síly v podobě hlubokých neuronových sítí. Neuronové sítě (dále NN - neural networks) jsou klasifikátory inspirované přírodou, respektive nervovou tkání. Jen pro připomenutí, ta je tvořena jednotlivými (vzájemně velmi podobnými) neurony, které na jedné straně přijímají signál od ostatních neuronů (pomocí různě širokých vstupních kanálů v dendritech, v NN šířkám kanálů říkáme váhy), signál mohou velmi jednoduchými operacemi upravit a při překročení kritické hladiny (prahu) signál transformovaný aktivační funkcí předat dalším neuronům.

![](images/neuron.svg)

Pro zjednodušení používáme typicky několik omezujících podmínek. Nejčastěji používané sítě jsou dopředné (feed-forward). Neuron tedy může posílat signál pouze směrem vpřed, do další vrstvy; což je druhé zjednodušení. Neurony organizujeme do takzvaných vrstev, kdy všechny neurony v jedné vrstvě mají stejný typ a aktivační funkci (pozor, ne váhy a práh, ty jsou učené nezávisle).

Jedinou drobnou komplikací je, že pro zpracování dat s kontextem je vhodné používat konvoluční neuronové sítě (CNN). Konvoluci (1D) snad nemusím připomínat, konvoluční vrstva je pak jen aplikací tohoto principu ("jedeme" okénkem po obrázku a vracíme okénkem váženou sumu intenzit pixelů).

Pro shrnutí, stačí znát několik základních pojmů:
- **vstupní vrstva** - vrstva neuronové sítě, která nemá žádné vstupy a výstupy těchto neuronů jsou nastaveny na hodnoty samotných dat
- **výstupní vrstva** - vrstva neuronové sítě, která nemá výstupy ale obsahuje po "protečení" dat sítí požadovanou výstupní hodnotu
- **plně propojená (fully connected) vrstva** - vrstva, ve které jsou všechny neurony napojené na všechny neurony předchozí vrstvy
- **konvoluční (convolutional) vrstva** - vrstva, která počítá konvoluci nad vstupními daty, trénují se tak pro každý výstupní filtr "jen" váhy konvolučních jader a jeden práh
- **sdružovací (pooling) vrstva** - vrstva redukující dimenzionalitu bez trénovatelných parametrů
- **softmax vrstva** - vrstva, které vektor reálných čísel transformuje na distribuci pravděpodobnosti

Postup učení umělých neuronových sítí není dobře možné shrnout do několikaminutového tutoriálu, tak si představme, že síť považujeme za velkou černou krabici ze které trčí mnoho set tisíc až milionů potenciometrů. Všechny parametry sítě (včetně "tvaru" sítě) jsou laděny tak, aby pro data na vstupu (trénovací množinu, v našem případě intenzity jednotlivých pixelů v jednotlivých kanálech) síť vracela požadovaný výstup (pravděpodobnosti tříd, tedy v našem případě jmen) a pro negativní příklady vracela nuly. Pro představu pomůže velmi dobře stravitelné video od CGP Grey: https://www.youtube.com/watch?v=wvWpdrfoEv0

Neuronová síť je jen tak dobrá, jak dobře je vyladěný její model. K tomu jsou potřeba obrovská množství dat. A model pak stejně rozpozná jen to, na co byl natrénován. Takže celou síť od začátku zde trénovat také nebudeme. Ale pokud by Vás zajímalo jaké modely jsou jednoduše dostupné pro PyTorch: http://www.robots.ox.ac.uk/~albanie/pytorch-models.html


### Příklad kovnoluční sítě — VGG


Síť VGG16 vypadá takto:
![](images/vgg16.png)

Upravená VGG — VGG-Face má v poslední, klasifikační vrstvě, 2622 neuronů, jejichž koncové hodnoty odpovídají pravděpodobnosti klasifikace do jmen.



### OpenFace
Tento přístup má jistě mnohá úskalí. Pokud se dotyčný nedívá přímo do kamery nebo pokud má výrazně nakloněnou hlavu, detekce ani popis nemusí fungovat správně. Poté je potřeba dělat trochu více magie, kterou nechcete psát stále dokola. A proto vznikla knihovna OpenFace: https://cmusatyalab.github.io/openface/

Doporučuji vyzkoušet.

# Pokračování o neurónových sítích — od základů až po anotaci obličeje

Začnime v [Tensorflow Playgroundu](https://playground.tensorflow.org/)

<div>
Pak základné pojmy, datasety a nástroje, jednoduché sítě na zpracování obrazu:
<a target="_blank" href="https://colab.research.google.com/drive/1sk_o6f-1hLSsJordNHL5nzMGtHqt_3u0"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
</div>

<div>
Mask-RCNN demo
    
<a target="_blank" href="https://colab.research.google.com/drive/1zmfeGCpmsyJB0S3aSp0769kW0-60NcuD"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
</div>

<div>
A na závěr pokročilé konvoluční sítě, framework Detectron2 a anotace obličeje:
<a target="_blank" href="https://colab.research.google.com/drive/1Ce7bXGvQxud-5xfE5YlDX_kbGzmusKje"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
</div>